In [1]:
import keras
import tensorflow as tf
import numpy
import pandas
import tensorflow_hub as hub

In [2]:
dataset = pandas.read_csv("ReadyDataSet.csv", index_col=False)
dataset = dataset.iloc[:, 1:]
dataset.head()

,X,target
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [3]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  df = dataframe.copy()
  labels = df.pop('target')
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

ds = df_to_dataset(dataset)
[(train_features, label_batch)] = ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of targets:', label_batch )


C:\Users\Kamil\anaconda3\envs\blue\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  after removing the cwd from sys.path.


Every feature: ['X', 'target']
A batch of targets: tf.Tensor([0 0 4 0 0 4 0 0 4 4 0 0 0 4 0 0 0 0 0 0 4 0 4 4 4 4 4 0 4 0 0 4], shape=(32,), dtype=int64)


In [4]:
from keras.layers import TextVectorization